In [2]:
import findspark
findspark.init('E:\spark\spark-3.3.1-bin-hadoop2/')
import pyspark
import seaborn as sns
import pandas as pd

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [4]:
mongo_ip = 'mongodb://localhost:27017/Cleaned_Insurance'

In [5]:
spark = SparkSession \
.builder\
.appName("myApp")\
.config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
.getOrCreate()

In [6]:
b_08 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",mongo_ip + ".ben_2008").load()

In [7]:
b_08.createOrReplaceTempView('b_08')

In [8]:
b_09 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",mongo_ip + ".ben_2009").load()

In [9]:
b_09.createOrReplaceTempView('b_09')

In [10]:
b_10 = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",mongo_ip + ".ben_2010").load()

In [11]:
b_10.createOrReplaceTempView('b_10')

In [12]:
inp = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",mongo_ip + ".Inpatient").load()

In [13]:
inp.createOrReplaceTempView('inp')

In [14]:
from pyspark.ml.feature import Bucketizer

boundaries = [25,30,45,60,75,90,105]
category = ['25-30','31-45','46-60','61-75','76-90','91-105']

# Create the bucketizer instance
bucketizer = Bucketizer(splits=boundaries, inputCol="Age", outputCol="Age_Group")

# Apply the bucketizer to the DataFrame
b_8 = bucketizer.transform(b_08)

In [15]:
b_8.createOrReplaceTempView('b_8')

In [16]:
from pyspark.sql.functions import regexp_replace
b_8 = b_8.withColumn('Age_Group', regexp_replace('Age_Group', '0.0', '25-30'))\
.withColumn('Age_Group', regexp_replace('Age_Group', '1.0', '31-45'))\
.withColumn('Age_Group', regexp_replace('Age_Group', '2.0', '46-60'))\
.withColumn('Age_Group', regexp_replace('Age_Group', '3.0', '61-75'))\
.withColumn('Age_Group', regexp_replace('Age_Group', '4.0', '76-90'))\
.withColumn('Age_Group', regexp_replace('Age_Group', '5.0', '91-105'))

In [17]:
b_8.createOrReplaceTempView('b_8')

In [18]:
spark.sql("select distinct Age_Group from b_8").show()

+---------+
|Age_Group|
+---------+
|    31-45|
|    25-30|
|    46-60|
|    76-90|
|   91-105|
|    61-75|
+---------+



In [19]:
import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
b_8.columns

['Age',
 'Alzheimer',
 'Cancer',
 'Chronic_Kidney_Disease',
 'Chronic_Obstructive_Pulmonary_Disease',
 'DESYNPUF_ID',
 'Date_of_birth',
 'Date_of_death',
 'Depression',
 'Diabetes',
 'End_stage_renal_disease_Indicator',
 'Gender',
 'Heart_Failure',
 'Inpatient_primary_payer_reimbursement_amount',
 'Inpatient_reimbursement_amount',
 'Inpatient_responsibility_amount',
 'Ischemic_Heart_Disease',
 'Osteoporosis',
 'Outpatient_primary_payer_reimbursement_amount',
 'Outpatient_reimbursement_amount',
 'Outpatient_responsibility_amount',
 'Race_Code',
 'Rheumatoid_arthritis_osteoarthritis',
 'State_Code',
 'Stroke_transient_Ischemic_Attack',
 'Total_months_HMO_coverage',
 'Total_months_partA_coverage',
 'Total_months_partB_coverage',
 'Total_months_partD_coverage',
 '_id',
 'total_number_of_Insurance',
 'total_number_of_diseases',
 'year',
 'Age_Group']

## State wise hotspots for diseases

In [21]:
spark.sql("select State_Code,sum(total_number_of_diseases) as Total_Diseases from b_8 group by State_Code order by Total_Diseases desc limit 5 ").show()

+----------+--------------+
|State_Code|Total_Diseases|
+----------+--------------+
|        05|         63781|
|        10|         56099|
|        45|         51776|
|        33|         47630|
|        39|         34508|
+----------+--------------+



## Average number of months for different type of coverage

spark.sql('select avg(Total_months_partA_coverage) as avg_month_A,avg(Total_months_partB_coverage) as avg_month_B , avg(Total_months_HMO_coverage) as avg_month_HMO , avg(Total_months_partD_coverage) as avg_month_D from b_8 ').show()

## State wise fraud claims

In [22]:
spark.sql('select b.State_Code,count(i.Claim_ID ) as fraud_claim_no from b_8 as b inner join inp as i on (b.DESYNPUF_ID = i.DESYNPUF_ID) where i.Claim_Payment_Amount > 0 AND b.Inpatient_reimbursement_amount == 0 group by b.State_Code order by fraud_claim_no desc limit 5').show()

+----------+--------------+
|State_Code|fraud_claim_no|
+----------+--------------+
|        05|          6814|
|        10|          5856|
|        45|          5165|
|        33|          4897|
|        14|          3503|
+----------+--------------+



In [23]:
spark.sql('select b.State_Code,count(i.Claim_ID ) as fraud_claim_no from b_09 as b inner join inp as i on (b.DESYNPUF_ID = i.DESYNPUF_ID) where i.Claim_Payment_Amount > 0 AND b.Inpatient_reimbursement_amount == 0 group by b.State_Code order by fraud_claim_no desc limit 5').show()

+----------+--------------+
|State_Code|fraud_claim_no|
+----------+--------------+
|        05|          5988|
|        10|          5612|
|        45|          5279|
|        33|          4821|
|        14|          3587|
+----------+--------------+



In [ ]:
spark.sql('select b.State_Code,count(i.Claim_ID ) as fraud_claim_no from b_10 as b inner join inp as i on (b.DESYNPUF_ID = i.DESYNPUF_ID) where i.Claim_Payment_Amount > 0 AND b.Inpatient_reimbursement_amount == 0 group by b.State_Code order by fraud_claim_no desc limit 5').show()

## Provider wise Fraud Claim

In [ ]:
spark.sql("select i.Provider_Number,count(i.Claim_ID ) as fraud_claim_no from b_8 b inner join inp i on (b.DESYNPUF_ID = i.DESYNPUF_ID) where i.Claim_Payment_Amount > 0 AND b.Inpatient_reimbursement_amount == 0 group by i.Provider_Number order by fraud_claim_no desc limit 5").show()

In [ ]:
spark.sql("select i.Provider_Number,count(i.Claim_ID ) as fraud_claim_no from b_09 b inner join inp i on (b.DESYNPUF_ID = i.DESYNPUF_ID) where i.Claim_Payment_Amount > 0 AND b.Inpatient_reimbursement_amount == 0 group by i.Provider_Number order by fraud_claim_no desc limit 5").show()

In [ ]:
spark.sql("select i.Provider_Number,count(i.Claim_ID ) as fraud_claim_no from b_10 b inner join inp i on (b.DESYNPUF_ID = i.DESYNPUF_ID) where i.Claim_Payment_Amount > 0 AND b.Inpatient_reimbursement_amount == 0 group by i.Provider_Number order by fraud_claim_no desc limit 5").show()

In [ ]:
inp.columns

## No. of Claims Weekly

In [40]:
ts = inp.select(inp['Claim_Start_Date'],inp['Claim_ID'])

In [41]:
ts.createOrReplaceTempView('ts')

In [26]:
MR = spark.sql('select Claim_Start_Date,count(Claim_ID) as no_of_claims_perday from ts group by Claim_Start_Date order by Claim_Start_Date')

In [ ]:
MR1 = spark.sql('select ')

In [27]:
tsp = MR.toPandas()

In [28]:
tsp.head(20)

,Claim_Start_Date,no_of_claims_perday
0,2007-11-27,3
1,2007-11-28,1
2,2007-11-29,2
3,2007-12-02,3
4,2007-12-03,1
5,2007-12-04,3
6,2007-12-05,2
7,2007-12-08,2
8,2007-12-09,2
9,2007-12-10,3


In [31]:
tsp.shape

(1126, 2)

In [35]:
tsp.to_csv('C:/Users/ASUS/Desktop/Insurance/claim_pred.csv')

In [36]:
tsp['Year'] = tsp['Claim_Start_Date'].dt.year
tsp['Month'] = tsp['Claim_Start_Date'].dt.month
tsp['quarter'] = tsp['Claim_Start_Date'].dt.quarter


In [37]:
month =tsp['Month'] = tsp['Claim_Start_Date'].dt.month

In [39]:
month.head(20)

0     11
1     11
2     11
3     12
4     12
5     12
6     12
7     12
8     12
9     12
10    12
11    12
12    12
13    12
14    12
15    12
16    12
17    12
18    12
19    12
Name: Claim_Start_Date, dtype: int64

In [172]:
tsp.head(20)

,Claim_Start_Date,no_of_claims_perday,Year,Month,quarter,Week,Week_of_Month
0,2007-11-27,3,2007,11,4,48,4
1,2007-11-28,1,2007,11,4,48,4
2,2007-11-29,2,2007,11,4,48,5
3,2007-12-02,3,2007,12,4,48,1
4,2007-12-03,1,2007,12,4,49,1
5,2007-12-04,3,2007,12,4,49,1
6,2007-12-05,2,2007,12,4,49,1
7,2007-12-08,2,2007,12,4,49,2
8,2007-12-09,2,2007,12,4,49,2
9,2007-12-10,3,2007,12,4,50,2


In [181]:
tsp.groupby(by=["Week"]).agg({'no_of_claims_perday' : 'sum'})

,no_of_claims_perday
Week,
1,3537
2,3738
3,3640
4,3789
5,3768
6,3725
7,3866
8,3840
9,3956


In [43]:
year =tsp['Year'] = tsp['Claim_Start_Date'].dt.year


In [45]:
tsp['Month_Year']= tsp['Claim_Start_Date'].dt.to_period('M')

In [46]:
tsp.head(20)

,Claim_Start_Date,no_of_claims_perday,Year,Month,quarter,Month_Year
0,2007-11-27,3,2007,11,4,2007-11
1,2007-11-28,1,2007,11,4,2007-11
2,2007-11-29,2,2007,11,4,2007-11
3,2007-12-02,3,2007,12,4,2007-12
4,2007-12-03,1,2007,12,4,2007-12
5,2007-12-04,3,2007,12,4,2007-12
6,2007-12-05,2,2007,12,4,2007-12
7,2007-12-08,2,2007,12,4,2007-12
8,2007-12-09,2,2007,12,4,2007-12
9,2007-12-10,3,2007,12,4,2007-12


In [47]:
month_uniq = tsp['Month_Year'].unique()

In [54]:
pred  =tsp.groupby(by=["Month_Year"]).agg({'no_of_claims_perday' : 'sum'})

In [57]:
pred.head()

,no_of_claims_perday
Month_Year,
2007-11,6
2007-12,616
2008-01,4233
2008-02,4979
2008-03,6407


In [ ]:
tsp.groupby(by=["Week"]).agg({'no_of_claims_perday' : 'sum'})